In [0]:
!pip install --upgrade pip

     |████████████████████████████████| 2.1 MB 4.2 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.2.4
    Uninstalling pip-21.2.4:
      Successfully uninstalled pip-21.2.4


Instalando o gdal manualmente

In [0]:
%sh
#!/bin/bash
sudo apt-get update
# Isso instala o gdal na máquina
sudo apt-get install -y gdal-bin libgdal-dev
sudo apt-get install -y libproj-dev

Hit:1 https://repos.azul.com/zulu/deb stable InRelease
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:3 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  default-libmysqlclient-dev gdal-data hdf5-helpers libaec-dev libaec0
  libarmadillo-dev libarmadillo9 libarpack2 libarpack2-dev libboost-dev
  libboost1.71-dev libcfitsio-dev libcfitsio-doc libcfitsio8 libcharls-dev
  libcharls2 libdap-dev libdap25 libdapclient6v5 libdapserver7v5
  libepsilon-dev libepsilon1 libfreexl-dev libfreexl1 libfyba-dev libfyba0
  libgdal26 libgeos-3.8.0 libgeos-c1v5 libgeos-dev libgeotiff-dev libgeotiff5
  libgif-dev libhdf4-0-alt libhdf4-alt-dev libhdf5-103 libhdf5-cpp-103
  libhdf5-dev libjson-c-dev li

os arquivos precisam estar com os nomes corretos

In [0]:
%sh
cd /usr/lib/python3/dist-packages/osgeo
mv _gdal.cpython-38-x86_64-linux-gnu.so _gdal.so
mv _gdalconst.cpython-38-x86_64-linux-gnu.so _gdalconst.so
mv _ogr.cpython-38-x86_64-linux-gnu.so _ogr.so
mv _osr.cpython-38-x86_64-linux-gnu.so _osr.so
mv _gdal_array.cpython-38-x86_64-linux-gnu.so _gdal_array.so
ls /usr/lib/python3/dist-packages/osgeo

__init__.py
__pycache__
_gdal.so
_gdal_array.so
_gdalconst.so
_gnm.cpython-38-x86_64-linux-gnu.so
_ogr.so
_osr.so
gdal.py
gdal_array.py
gdalconst.py
gdalnumeric.py
gnm.py
ogr.py
osr.py


Bibliotecas

In [0]:
from pyspark.sql import SparkSession
import os

In [0]:
!pip install pyspark raster2xyz

     

*** WARNING: max output size exceeded, skipping output. ***

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 938.4 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ | / - \ | / done
  Preparing metadata (setup.py) ... - \ | / - \ done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 23.0 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488511 sha256=6aea876959edfc8267cce7e682df5511a2f7ebc9938b09a9846a1016d4b0dade
  Stored in directory: /root/.cache/pip/wheels/92/09/11/aa01d01a7f005fda8a66ad71d2be7f8aa341bddafb27eee3c7
  Created wheel for raster2xyz: filename=raster2xyz-0.1.3-py3-none-any.whl size=5965 sha256=7addcf123ad33f1f4239af06602ffb5020af5df49fdf90f3f35338ef2bb68298
  Stored in directory: /root/.cache/pip/wheels/c2/09/bb/e65a67b3120261e7517662ad9bcaa1b7c505bb981f5c6152b7
Successfully built pyspark raster2xyz


Iniciando sessão spark

In [0]:
spark=SparkSession.builder \
    .master('local[*]') \
    .getOrCreate()

Função para avaliar o benchmark

In [0]:
def benchmark(arquivo):
    df = spark.read.format("csv").option("header", True).load(arquivo)
    df.write.format("noop").mode("overwrite").save()

In [0]:
def partitions(arquivo):
  df = spark.read.format("csv").option("header", True).load(arquivo)
  print(f"Número de partições -> {df.rdd.getNumPartitions()}")

#Precipitação

##Precipitação de 10 minutos

In [0]:
!wget https://biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_10m_tavg.zip
!unzip -o wc2.1_10m_tavg.zip

--2024-05-04 23:47:23--  https://biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_10m_tavg.zip
Resolving biogeo.ucdavis.edu (biogeo.ucdavis.edu)... 128.120.228.172
Connecting to biogeo.ucdavis.edu (biogeo.ucdavis.edu)|128.120.228.172|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://data.biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_10m_tavg.zip [following]
--2024-05-04 23:47:23--  https://data.biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_10m_tavg.zip
Resolving data.biogeo.ucdavis.edu (data.biogeo.ucdavis.edu)... 128.120.228.172
Connecting to data.biogeo.ucdavis.edu (data.biogeo.ucdavis.edu)|128.120.228.172|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 37364656 (36M) [application/zip]
Saving to: ‘wc2.1_10m_tavg.zip’

wc2.1_10m_tavg.zip  100%[===================>]  35.63M  31.4MB/s    in 1.1s    

2024-05-04 23:47:24 (31.4 MB/s) - ‘wc2.1_10m_tavg.zip’ saved [37364656/37364656]

Archive:  wc2.1_10m_tavg.zip
  infl

In [0]:
!raster2xyz wc2.1_10m_tavg_01.tif wc2.1_10m_tavg_01.csv

[2024-05-04 11:47:29 - INFO] - Getting geotransform and data...
[2024-05-04 11:47:29 - INFO] - Getting XYZ data...
[2024-05-04 11:47:29 - INFO] - Getting geotransformed coordinates...
[2024-05-04 11:47:30 - INFO] - Building XYZ data...
[2024-05-04 11:47:39 - INFO] - New XYZ (csv file) created...


Os arquivos são salvos no cluster então é necessário movê-los para o dbfs

In [0]:
dbutils.fs.cp("file:/databricks/driver/wc2.1_10m_tavg_01.csv", "dbfs:/FileStore/tables/wc2.1_10m_tavg_01.csv")

Out[11]: True

Testando o benchmark

In [0]:
arquivo='dbfs:/FileStore/tables/wc2.1_10m_tavg_01.csv'

In [0]:
partitions(arquivo)

Número de partições -> 8


In [0]:
for i in range(1,33):
 %time benchmark(arquivo)

CPU times: user 13.2 ms, sys: 3.56 ms, total: 16.8 ms
Wall time: 14.8 s
CPU times: user 7.97 ms, sys: 3.36 ms, total: 11.3 ms
Wall time: 9.7 s
CPU times: user 7.21 ms, sys: 3.87 ms, total: 11.1 ms
Wall time: 8.82 s
CPU times: user 9.92 ms, sys: 934 µs, total: 10.9 ms
Wall time: 7.49 s
CPU times: user 9.36 ms, sys: 844 µs, total: 10.2 ms
Wall time: 7.94 s
CPU times: user 5.83 ms, sys: 5.09 ms, total: 10.9 ms
Wall time: 7.53 s
CPU times: user 11.5 ms, sys: 0 ns, total: 11.5 ms
Wall time: 7.91 s
CPU times: user 7.31 ms, sys: 3.29 ms, total: 10.6 ms
Wall time: 7.64 s
CPU times: user 4.67 ms, sys: 4.72 ms, total: 9.39 ms
Wall time: 6.91 s
CPU times: user 10.2 ms, sys: 477 µs, total: 10.7 ms
Wall time: 7.95 s
CPU times: user 11.7 ms, sys: 3.16 ms, total: 14.9 ms
Wall time: 6.37 s
CPU times: user 5.71 ms, sys: 3.89 ms, total: 9.6 ms
Wall time: 6.75 s
CPU times: user 8.48 ms, sys: 712 µs, total: 9.19 ms
Wall time: 6.76 s
CPU times: user 8.69 ms, sys: 746 µs, total: 9.44 ms
Wall time: 6.15 s
CP

##Precipitação 5 minutos

In [0]:
!wget https://biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_5m_tavg.zip
!unzip -o wc2.1_5m_tavg.zip

--2024-05-04 23:51:56--  https://biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_5m_tavg.zip
Resolving biogeo.ucdavis.edu (biogeo.ucdavis.edu)... 128.120.228.172
Connecting to biogeo.ucdavis.edu (biogeo.ucdavis.edu)|128.120.228.172|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://data.biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_5m_tavg.zip [following]
--2024-05-04 23:51:56--  https://data.biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_5m_tavg.zip
Resolving data.biogeo.ucdavis.edu (data.biogeo.ucdavis.edu)... 128.120.228.172
Connecting to data.biogeo.ucdavis.edu (data.biogeo.ucdavis.edu)|128.120.228.172|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 126719768 (121M) [application/zip]
Saving to: ‘wc2.1_5m_tavg.zip’

wc2.1_5m_tavg.zip   100%[===================>] 120.85M  39.4MB/s    in 3.2s    

2024-05-04 23:52:00 (37.8 MB/s) - ‘wc2.1_5m_tavg.zip’ saved [126719768/126719768]

Archive:  wc2.1_5m_tavg.zip
  inflat

In [0]:
!raster2xyz wc2.1_5m_tavg_01.tif wc2.1_5m_tavg_01.csv

[2024-05-04 11:52:04 - INFO] - Getting geotransform and data...
[2024-05-04 11:52:04 - INFO] - Getting XYZ data...
[2024-05-04 11:52:04 - INFO] - Getting geotransformed coordinates...
[2024-05-04 11:52:05 - INFO] - Building XYZ data...
[2024-05-04 11:52:45 - INFO] - New XYZ (csv file) created...


In [0]:
dbutils.fs.cp("file:/databricks/driver/wc2.1_5m_tavg_01.csv", "dbfs:/FileStore/tables/wc2.1_5m_tavg_01.csv")

Out[17]: True

In [0]:
arquivo='dbfs:/FileStore/tables/wc2.1_5m_tavg_01.csv'

In [0]:
partitions(arquivo)

Número de partições -> 8


In [0]:
for i in range(1,33):
 %time benchmark(arquivo)

CPU times: user 11 ms, sys: 489 µs, total: 11.5 ms
Wall time: 15.6 s
CPU times: user 9.56 ms, sys: 7.58 ms, total: 17.1 ms
Wall time: 16.8 s
CPU times: user 8.2 ms, sys: 6.72 ms, total: 14.9 ms
Wall time: 18.2 s
CPU times: user 15.2 ms, sys: 3.4 ms, total: 18.6 ms
Wall time: 16.2 s
CPU times: user 9.73 ms, sys: 4.26 ms, total: 14 ms
Wall time: 17.1 s
CPU times: user 9.35 ms, sys: 4.31 ms, total: 13.7 ms
Wall time: 17.1 s
CPU times: user 10.5 ms, sys: 3.11 ms, total: 13.6 ms
Wall time: 17.1 s
CPU times: user 13.2 ms, sys: 866 µs, total: 14.1 ms
Wall time: 17.5 s
CPU times: user 11.6 ms, sys: 2.71 ms, total: 14.3 ms
Wall time: 16.5 s
CPU times: user 9.54 ms, sys: 4.89 ms, total: 14.4 ms
Wall time: 17.5 s
CPU times: user 8.38 ms, sys: 7.38 ms, total: 15.8 ms
Wall time: 17.9 s
CPU times: user 7.74 ms, sys: 6.61 ms, total: 14.3 ms
Wall time: 17.1 s
CPU times: user 12.7 ms, sys: 1.42 ms, total: 14.2 ms
Wall time: 15.9 s
CPU times: user 9.83 ms, sys: 4.28 ms, total: 14.1 ms
Wall time: 16.8 s


##Precipitação 2.5

In [0]:
!wget https://biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_2.5m_tavg.zip
!unzip -o wc2.1_2.5m_tavg.zip

--2024-05-05 00:02:05--  https://biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_2.5m_tavg.zip
Resolving biogeo.ucdavis.edu (biogeo.ucdavis.edu)... 128.120.228.172
Connecting to biogeo.ucdavis.edu (biogeo.ucdavis.edu)|128.120.228.172|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://data.biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_2.5m_tavg.zip [following]
--2024-05-05 00:02:05--  https://data.biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_2.5m_tavg.zip
Resolving data.biogeo.ucdavis.edu (data.biogeo.ucdavis.edu)... 128.120.228.172
Connecting to data.biogeo.ucdavis.edu (data.biogeo.ucdavis.edu)|128.120.228.172|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 443224846 (423M) [application/zip]
Saving to: ‘wc2.1_2.5m_tavg.zip’

wc2.1_2.5m_tavg.zip 100%[===================>] 422.69M  39.6MB/s    in 11s     

2024-05-05 00:02:16 (38.1 MB/s) - ‘wc2.1_2.5m_tavg.zip’ saved [443224846/443224846]

Archive:  wc2.1_2.5m_tavg.

In [0]:
!raster2xyz wc2.1_2.5m_tavg_01.tif wc2.1_2.5m_tavg_01.csv

[2024-05-05 12:02:24 - INFO] - Getting geotransform and data...
[2024-05-05 12:02:25 - INFO] - Getting XYZ data...
[2024-05-05 12:02:26 - INFO] - Getting geotransformed coordinates...
[2024-05-05 12:02:28 - INFO] - Building XYZ data...
[2024-05-05 12:05:06 - INFO] - New XYZ (csv file) created...


In [0]:
dbutils.fs.cp("file:/databricks/driver/wc2.1_2.5m_tavg_01.csv", "dbfs:/FileStore/tables/wc2.1_2.5m_tavg_01.csv")

Out[23]: True

In [0]:
arquivo='dbfs:/FileStore/tables/wc2.1_2.5m_tavg_01.csv'


In [0]:
partitions(arquivo)

Número de partições -> 11


In [0]:
for i in range(1,33):
 %time benchmark(arquivo)

CPU times: user 22.1 ms, sys: 16.4 ms, total: 38.5 ms
Wall time: 57.4 s
CPU times: user 23.2 ms, sys: 10.4 ms, total: 33.5 ms
Wall time: 57.6 s
CPU times: user 23.3 ms, sys: 9.27 ms, total: 32.5 ms
Wall time: 57.2 s
CPU times: user 15 ms, sys: 16.6 ms, total: 31.5 ms
Wall time: 56.9 s
CPU times: user 19.7 ms, sys: 12.6 ms, total: 32.4 ms
Wall time: 57.1 s
CPU times: user 20.9 ms, sys: 12.2 ms, total: 33.1 ms
Wall time: 57.9 s
CPU times: user 24.3 ms, sys: 9.35 ms, total: 33.7 ms
Wall time: 56.1 s
CPU times: user 23.2 ms, sys: 9.18 ms, total: 32.4 ms
Wall time: 59.2 s
CPU times: user 30 ms, sys: 2.22 ms, total: 32.2 ms
Wall time: 58 s
CPU times: user 27.3 ms, sys: 5.17 ms, total: 32.5 ms
Wall time: 56.2 s
CPU times: user 17.6 ms, sys: 15.1 ms, total: 32.6 ms
Wall time: 57.4 s
CPU times: user 19.1 ms, sys: 20.1 ms, total: 39.2 ms
Wall time: 55.1 s
CPU times: user 26.9 ms, sys: 3.95 ms, total: 30.9 ms
Wall time: 55.2 s
CPU times: user 9.2 ms, sys: 24.4 ms, total: 33.6 ms
Wall time: 57.1 s

##Precipitação de 30 segundos

In [0]:
!wget https://biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_30s_tavg.zip
!unzip -o wc2.1_30s_tavg.zip

--2024-05-05 00:36:59--  https://biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_30s_tavg.zip
Resolving biogeo.ucdavis.edu (biogeo.ucdavis.edu)... 128.120.228.172
Connecting to biogeo.ucdavis.edu (biogeo.ucdavis.edu)|128.120.228.172|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://data.biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_30s_tavg.zip [following]
--2024-05-05 00:37:00--  https://data.biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_30s_tavg.zip
Resolving data.biogeo.ucdavis.edu (data.biogeo.ucdavis.edu)... 128.120.228.172
Connecting to data.biogeo.ucdavis.edu (data.biogeo.ucdavis.edu)|128.120.228.172|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4299762045 (4.0G) [application/zip]
Saving to: ‘wc2.1_30s_tavg.zip’

wc2.1_30s_tavg.zip   36%[======>             ]   1.45G  39.7MB/s    eta 67s    

In [0]:
#arquivo mt grande raster não conseguiu rodar
!raster2xyz wc2.1_30s_tavg_01.tif wc2.1_30s_tavg_01.csv

[2024-05-03 05:31:34 - INFO] - Getting geotransform and data...
[2024-05-03 05:31:39 - INFO] - Getting XYZ data...
